In [1]:
"""
Constant-pressure, adiabatic kinetics simulation.

Requires: cantera >= 2.5.0, matplotlib >= 2.0
"""

import sys
import os
import numpy   as np

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

import cantera as ct

from scipy.integrate import solve_ivp

In [8]:
##########################################################################################
### Input Data

OutputDir          = WORKSPACE_PATH + '/ROMNet/Data_100DReact/'
FigDir             = OutputDir + '/fig/'

MixtureFile        = 'gri30.yaml'
NTs                = 10
T0Vec              = np.logspace(np.log10(300), np.log10(3000), NTs) # [2.e-5]
NPerT0             = 2000
tMinVec            = [5.e-3, 5.e-7]

tStratch           = 1.
Nt                 = NPerT0*2

P0Inlet            = ct.one_atm
EqRatioInlet       = 1.
##########################################################################################

In [9]:
def IdealGasConstPressureReactor_SciPY(t, y):
    print(t)

    YEnd     = np.array([1.-np.sum(y[1:])], dtype=np.float64)
    Y        = np.concatenate((y[1:], YEnd), axis=0)
    gas_.TPY = y[0], P_, Y
    
    wdot     = gas_.net_production_rates

    ydot     = np.zeros_like(y, dtype=np.float64)
    ydot[0]  = - np.dot(wdot, gas_.partial_molar_enthalpies) / gas_.cp / gas_.density
    ydot[1:] = wdot[0:-1] * gas_.molecular_weights[0:-1] / gas_.density
    
    return ydot


def IdealGasReactor_SciPY(t, y):
    print(t)

    YEnd     = np.array([1.-np.sum(y[1:])], dtype=np.float64)
    Y        = np.concatenate((y[1:], YEnd), axis=0)
    gas_.TDY = y[0], density_, Y
    
    wdot     = gas_.net_production_rates

    ydot     = np.zeros_like(y, dtype=np.float64)
    ydot[0]  = - np.dot(wdot, gas_.partial_molar_int_energies) / gas_.cv / density_
    ydot[1:] = wdot[0:-1] * gas_.molecular_weights[0:-1] / density_
    
    return ydot

In [10]:
gas     = ct.Solution('gri30.cti')
gas.TPX = 1250, ct.one_atm, 'CH4:0.5, O2:1, N2:3.76'

In [11]:
T0      = 1400
gas.TPX = T0, 10.*ct.one_atm, 'CH4:0.5, O2:1, N2:3.76'
r       = ct.IdealGasReactor(gas)
sim     = ct.ReactorNet([r])

gas_    = gas
mass_   = r.mass
density_= r.density

y0      = np.array(np.hstack((gas_.T, gas_.Y[0:-1])), dtype=np.float64)

In [16]:
tMin     = (tMinVec[1]-tMinVec[0])*(T0-900.)/(1700.-900.) + tMinVec[0]
tMax     = tMin*5.e2
dt0      = tMin*5.e-2
tStratch = 1.01
tVec     = [0.0]
t        = tMin
dt       = dt0
while (t <= tMax):
    tVec.append(t)
    t  =   t + dt
    dt = dt0 * tStratch
rtol       = 1.e-8
SOLVER     = 'BDF'#'RK23'#'BDF'#'Radau'

In [17]:
states     = ct.SolutionArray(gas, 1, extra={'t': [0.0]})
output     = solve_ivp( IdealGasReactor_SciPY, (tVec[0],tVec[-1]), y0, method=SOLVER, t_eval=tVec, rtol=rtol )

0.0
0.007754174465464937
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
9.98806121778412e-09
9.98806121778412e-09
1.997612243556824e-08
1.997612243556824e-08
1.1985673461340943e-07
1.1985673461340943e-07
2.1973734679125063e-07
2.1973734679125063e-07
1.2185434685696627e-06
1.2185434685696627e-06
2.2173495903480747e-06
2.2173495903480747e-06
6.557120953189e-06
6.557120953189e-06
6.557120953189e-06
1.0896892316029924e-05
1.0896892316029924e-05
1.0896892316029924e-05
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.650953929187936e-06
9.6509539

0.0002788385855056388
0.0002788385855056388
0.0002812878968862318
0.0002812878968862318
0.0002837372082668248
0.0002837372082668248
0.0002861865196474178
0.0002861865196474178
0.0002908142509381159
0.0002908142509381159
0.00029544198222881404
0.00029544198222881404
0.00030006971351951217
0.00030006971351951217
0.0003046974448102103
0.0003046974448102103
0.0003157754322566295
0.0003157754322566295
0.0003157754322566295
0.0003268534197030487
0.0003268534197030487
0.0003268534197030487
0.0003228088123867646
0.0003228088123867646
0.00032984219251689967
0.00032984219251689967
0.00033687557264703476
0.00033687557264703476
0.00034390895277716985
0.00034390895277716985
0.00035094233290730494
0.00035094233290730494
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.0003574365821579294
0.000357436582157929

0.0004012457896403839
0.0003990167633303893
0.0003990167633303893
0.00040124578964038395
0.00040124578964038395
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.00040347481595037857
0.000403474815

0.00042259572512048265
0.00042259572512048265
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.00042380001676556875
0.000423800

0.000445133924433006
0.000445133924433006
0.00044560264469346904
0.00044560264469346904
0.00044607136495393207
0.00044607136495393207
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.0004465400852143951
0.000446

...

0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.0008535592426759438
0.00085355

0.01055509258406529
0.01055509258406529
0.010070035068463504
0.010070035068463504
0.00982750631066261
0.00982750631066261
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463502
0.010070035068463

0.06342797127910052
0.06342797127910052
0.06342794574389146
0.06342794574389146
0.06342793297628693
0.06342793297628693
0.06342792659248467
0.06342792659248467
0.06342793297628692
0.06342793297628692
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943
0.06342799681430943


0.06343691077486355
0.06343691077486355
0.0634371542975933
0.0634371542975933
0.06343739782032307
0.06343739782032307
0.06343983304762066
0.06343983304762066
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.06344226827491825
0.

CanteraError: 
***********************************************************************
CanteraError thrown by Phase::setTemperature:
temperature must be positive. T = -58918.686620908746
***********************************************************************


In [ ]:
### Integrate
time         = []
temperature  = []
mfrac        = []
HR           = []
it           = 0
for t in output.t:
    current_time = t
    time.append(current_time)
    temperature.append(output.y[0,it])
    YY       = np.concatenate((output.y[1:,it], [1.0-np.sum(output.y[1:,it])]), axis=0)
    gas_.TDY = output.y[0,it], density_, YY
    mfrac.append(YY)
    HR.append(- np.dot(gas_.net_production_rates, gas_.partial_molar_enthalpies))
    it+=1 
    
auto_ignition = time[HR.index(max(HR))]
Y             = np.vstack(mfrac)

print('Auto Ignition Delay = ', auto_ignition)

In [ ]:
plt.plot(time, Y[:, gas.species_index('H2')])
plt.xlabel('Time (s)')
plt.ylabel('Temperature (K)')
plt.xscale('log')
#plt.xlim([1.e-6, 1.e-2])

In [ ]:
plt.plot(time, temperature)
plt.xlabel('Time (s)')
plt.ylabel('Temperature (K)')
plt.xscale('log')
#plt.xlim([1.e-6, 1.e-2])

In [ ]:
temperature[-1]

In [ ]:
EndSpecies = [gas.species_names[i] for i in range(gas.n_species) if (Y[900,i]*100 > .01)]

In [ ]:
EndSpecies

In [ ]:
len(temperature)